In [13]:
# Ingest data

In [5]:
import os
import glob
import pandas as pd

In [6]:
def pm25_template_table(date):
    row_list = []
    columns = ['เขต', 'สถานีตรวจวัด', 'วันที่และเวลา', 'PM10', 'PM2.5', 'CO', 'NO2',
                'O3', 'Wind Speed', 'Wind Direction', 'Temperature',
                'Relative Humidity', 'Air Pressure']
    for i in range(24):
        datetime_str = f"{date} {str(i).zfill(2)}:00"
        row = [None,None,datetime_str] + [None for i in range(len(columns)-3)]
        row_list.append(row)
    return pd.DataFrame(row_list, columns=columns)


In [ ]:
# Get all CSV files recursively
csv_files = glob.glob(f"../data/เขตคลองเตย/**/*.csv", recursive=True)

# Ensure there are CSV files before proceeding
if not csv_files:
    print("No CSV files found. Exiting.")
    exit()

# Load all CSV files into a single DuckDB DataFrame
df_list = []
for file in csv_files:
    file_empthy = False
    try:
        temp_df = pd.read_csv(file)
        if temp_df.empty:
            file_empthy = True
    except:
        file_empthy = True

    if file_empthy:
        date = file.split('/')[-1][:10]
        year,month,day = date.split('-')
        date_re = f'{int(day)}/{int(month)}/{year}'
        temp_df = pm25_template_table(date_re)

    df_list.append(temp_df)

df = pd.concat(df_list, axis=0) # Warning occurs since df contain all-NA columns

/tmp/ipykernel_75868/791464698.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, axis=0)


In [13]:
df.columns = ['district', 'station', 'datetime_str'] + df.columns[3:].to_list()

In [17]:
df['datetime'] = pd.to_datetime(df['datetime_str'], format="%d/%m/%Y %H:%M")

In [19]:
df = df.drop(columns=['PM10','CO'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16128 entries, 0 to 23
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   district           16008 non-null  object        
 1   station            16008 non-null  object        
 2   datetime_str       16128 non-null  object        
 3   PM2.5              14816 non-null  float64       
 4   NO2                15537 non-null  float64       
 5   O3                 15629 non-null  float64       
 6   Wind Speed         15170 non-null  float64       
 7   Wind Direction     15170 non-null  float64       
 8   Temperature        15170 non-null  float64       
 9   Relative Humidity  15170 non-null  float64       
 10  Air Pressure       15148 non-null  float64       
 11  datetime           16128 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 1.6+ MB


In [ ]:
# Fill in missing data
# 1. district and station --> corresponding to others since indentical
# 2. Numerical values --> interpolate